In [27]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset
from torchvision import datasets
from PIL import Image
import glob

In [28]:
class AnimalDataset(Dataset):
    def __init__(self, TrainTest="train"):
        super().__init__()
        """
        どのデータを使うのかを記述する部分
        """
        #ここではtrain/testのみのスプリットをしておき，valは別機能で外からスプリットする
        if TrainTest == "train":
            #self.img_path_list = sorted(glob.glob("/content/drive/MyDrive/segmentation/data/train/org/*"))
            self.img_path_list = sorted(glob.glob("/takaya_workspace/self_study/deep_learning/classification_sample/data/train/*/*"))
        elif TrainTest == "test":
            self.img_path_list = sorted(glob.glob("/takaya_workspace/self_study/deep_learning/classification_sample/data/test/*/*"))
    
    def __len__(self):
        """
        データがいくつあるのかを数える
        """
        return len(self.img_path_list)
    
    def __getitem__(self, index):
        """
        データをどのような形で取り出すのか記述する
        """
        image_path = self.img_path_list[index] # ファイル名
        if "dog" in image_path:
            label = np.array(0)
        else:
            label = np.array(1)
        
        img = Image.open(image_path) # ファイル名を与えて画像を取り出す
        img = np.array(img) # 画像をnumpy形式の行列へ変換
        img = img.transpose(2,0,1)
        img = torch.tensor(img) # 行列をpytorchで扱える形式（tensor型）に変換する
        img = img / 255 # 0~255までの値を0~1までの値に変換する
        
    
        label = torch.tensor(label)
        
        return img, label

In [42]:
# transformの定義
from torchvision import transforms
data_transforms = transforms.Compose([transforms.ToTensor()])

In [43]:
#animal_train = AnimalDataset(TrainTest="train")
#animal_test = AnimalDataset(TrainTest="test")
animal_train = datasets.ImageFolder("/takaya_workspace/self_study/deep_learning/classification_sample/data/train/", data_transforms)

In [44]:
n_samples = len(animal_train) 
train_size = int(n_samples * 0.8) 
val_size = n_samples - train_size 

train_dataset, val_dataset = torch.utils.data.random_split(animal_train, [train_size, val_size])

In [45]:
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)

In [46]:
len(train_loader)

24

In [53]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18

class resnet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.feature = resnet18(pretrained=True)
        self.fc = nn.Linear(1000, 2)
    
    def forward(self, x):
        h = self.feature(x)
        h = self.fc(h)
        return h
    
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = torch.nn.Conv2d(3,  # チャネル入力
#                                      60,  # チャンネル出力
#                                      3,  # カーネルサイズ
#                                      1,  # ストライド (デフォルトは1)
#                                      0,  # パディング (デフォルトは0)
#                                      )
#         self.conv2 = torch.nn.Conv2d(60, 30, 3)
 
#         self.pool = torch.nn.MaxPool2d(2, 2)  # カーネルサイズ, ストライド
 
#         self.fc1 = torch.nn.Linear(87480, 500)  # 入力サイズ, 出力サイズ
#         self.fc2 = torch.nn.Linear(500, 100)
#         self.fc3 = torch.nn.Linear(100, 2)
 
#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = self.pool(x)
#         x = F.relu(self.conv2(x))
#         x = self.pool(x)
#         x = x.view(x.size(0), -1)  # 1次元データに変えて全結合層へ
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
        
#         return x

In [77]:
from torch.nn import BCELoss
import torch.optim as optim

# モデルの定義
model = resnet()

# GPUを使う場合は，下記のコメントを外す
model = model.to("cuda")

# optimizer（勾配降下法のアルゴリズム）の準備
optimizer = optim.RAdam(model.parameters())

# 誤差関数の定義
criterion = nn.CrossEntropyLoss()

# 学習ループ
epochs = 5 #ミニバッチのサンプリングが一巡 = 1 epoch

train_loss_list = [] # epoch毎のtrain_lossを保存しておくための入れ物
val_loss_list = [] # epoch毎のvalidation_lossを保存しておくための入れ物

loss_min = 100000 # validation_lossが小さくなった場合にのみモデルを保存しておくためのメモ

# ここからループ開始
for epoch in range(epochs):
    
    train_loss_add = 0 # 1エポック分の誤差を累積しておくための変数
    
    model.train() #学習モードであることを明示
   
    for i, data in enumerate(train_loader):
        
        x, t = data # ①データの読み込み
        
        #GPU環境で動かす際は，下記2行のコメントを外す
        x = x.to("cuda")
        t = t.to("cuda")
        predict = model(x) # ②順伝播計算
        
        loss = criterion(predict, t) # ③誤差の計算
        
        model.zero_grad()# 誤差逆伝播法のための準備
        loss.backward() # ④誤差逆伝播法による誤差の計算
        
        optimizer.step() # ⑤勾配を用いてパラメータを更新
        
        train_loss_add += loss.data # あとで平均を計算するために，誤差を累積しておく
        
    train_loss_mean = train_loss_add / int(len(train_dataset)/train_loader.batch_size) # 1epochでの誤差の平均を計算
    print("epoch" + str(epoch+1))
    print("train_loss:" + str(train_loss_mean))
    train_loss_list.append(train_loss_mean.cpu())# 1epoch毎の平均を格納しておく
    
    # validation
    model.eval() # 評価モード（学習を行わない）であることを明示
    
    val_loss_add = 0
    for i, data in enumerate(val_loader):
            
        x, t = data
        
        #GPU環境で動かす際は，下記2行のコメントを外す
        x = x.to("cuda")
        t = t.to("cuda")
        
        predict = model(x) # 順伝播計算
        
        loss = criterion(predict, t) # 誤差の計算
        val_loss_add += loss.data
        
    val_loss_mean = val_loss_add / int(len(val_dataset)/val_loader.batch_size)
    print("val_loss:" + str(val_loss_mean))
    val_loss_list.append(val_loss_mean.cpu())
    
    if val_loss_mean < loss_min: # 前に保存したモデルよりもvalidation lossが小さければ，モデルを保存する
        torch.save(model.state_dict(), "/takaya_workspace/self_study/deep_learning/classification_sample/best.model")
        print("saved best model!")
        loss_min = val_loss_mean # 今回保存したモデルのvalidation lossをメモしておく

epoch1
train_loss:tensor(0.4541, device='cuda:0')
val_loss:tensor(0.0371, device='cuda:0')
saved best model!
epoch2
train_loss:tensor(0.1903, device='cuda:0')
val_loss:tensor(0.0288, device='cuda:0')
saved best model!
epoch3
train_loss:tensor(0.1297, device='cuda:0')
val_loss:tensor(0.3790, device='cuda:0')
epoch4
train_loss:tensor(0.1100, device='cuda:0')
val_loss:tensor(0.0605, device='cuda:0')
epoch5
train_loss:tensor(0.2853, device='cuda:0')
val_loss:tensor(0.3197, device='cuda:0')


In [58]:
from glob import glob
from natsort import natsorted

paths = natsorted(glob("/takaya_workspace/self_study/deep_learning/classification_sample/data/test/*.jpg"))

In [61]:
imgs = []
for path in paths:
    img = Image.open(path)
    img = data_transforms(img)
    imgs.append(img)

In [63]:
len(imgs)

200

In [64]:
imgs = torch.stack(imgs)

In [65]:
imgs.shape

torch.Size([200, 3, 224, 224])

In [78]:
imgs = imgs.to("cuda")

model_path = "/takaya_workspace/self_study/deep_learning/classification_sample/best.model"

model.load_state_dict(torch.load(model_path)) # 学習時に保存しておいたモデルのパラメータをコピー

y = model(imgs)

In [79]:
results = np.array(y.argmax(axis=1).cpu())

In [80]:
results = np.array(y.argmax(axis=1).cpu())
import pandas as pd
pd.Series(results, name="class").to_csv("submission.csv", index=None)

In [82]:
for i, idx in enumerate(range(0, 100, 10)):
    print(idx)

0
10
20
30
40
50
60
70
80
90
